In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchinfo import summary

import os
import glob
import time
import datetime as dt
from netCDF4 import Dataset as nc_Dataset
from netCDF4 import date2num, num2date
import pandas as pd
import numpy as np
import math
import xarray as xr
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from HRRR_URMA_Datasets_AllVars import *
from DefineModelAttributes import *
from SR_UNet_simple import *
from utils import *

In [2]:
TRAINING_LOG_FILEPATH = "/scratch/RTMA/alex.schein/CNN_Main/Training_logs/training_log.txt"

NUM_GPUS_TO_USE = 3

BATCH_SIZE = 256*NUM_GPUS_TO_USE #Be careful when initializing with this; should use BATCH_SIZE in DataLoader but use BATCH_SIZE/NUM_GPUS in model setup call
NUM_EPOCHS = 1000

PREDICTOR_VAR_LIST = ["t2m", "d2m", "pressurf", "u10m", "v10m"]
TARGET_VAR_LIST = [["u10m", "v10m"]]#["d2m", "pressurf", "u10m", "v10m", ["u10m", "v10m"]] #don't need t2m, already have that. Also want a model that targets both u and v at the same time, but only those vars

In [3]:
for i in range(len(TARGET_VAR_LIST)):
    current_model = DefineModelAttributes(predictor_vars=PREDICTOR_VAR_LIST,
                                         target_vars=(TARGET_VAR_LIST[i] if isinstance(TARGET_VAR_LIST[i],list) else [TARGET_VAR_LIST[i]]))

    current_model.create_dataset()
    current_model_dataloader = DataLoader(current_model.dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4*NUM_GPUS_TO_USE, pin_memory=True)

Terrain normalization done for ['diff']
Loading predictor dataset for t2m (Years = 2021/22/23, months = 1 to 12, hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
Predictor dataset data loaded. Time taken = 5.8 sec
Normalizing over all times
Normalization done. Time taken = 1.6 sec
Loading predictor dataset for d2m (Years = 2021/22/23, months = 1 to 12, hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
Predictor dataset data loaded. Time taken = 5.3 sec
Normalizing over all times
Normalization done. Time taken = 1.6 sec
Loading predictor dataset for pressurf (Years = 2021/22/23, months = 1 to 12, hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
Predictor dataset data loaded. Time taken = 5.1 sec
Normalizing over all times
Normalization done. Time taken = 1.6 sec
Loading predictor dataset for u10m (Years = 2021/22/23, months = 1 to 12, hours = [0, 1,

In [4]:
np.shape(current_model.dataset[0][1])[0]

2

In [11]:
model = SR_UNet_simple(n_channels_in=current_model.num_channels_in, n_channels_out=current_model.num_channels_out)
device = torch.device("cuda")
model = nn.DataParallel(model, device_ids=[i for i in range(NUM_GPUS_TO_USE)])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=[0.5,0.999]) #torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_function = torch.nn.L1Loss()

model.train()

DataParallel(
  (module): SR_UNet_simple(
    (first_conv): FirstConv(
      (conv): Sequential(
        (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), padding=same, bias=False)
        (1): LeakyReLU(negative_slope=0.1, inplace=True)
      )
    )
    (second_conv): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
        (1): LeakyReLU(negative_slope=0.1, inplace=True)
      )
    )
    (enc_1): Encoder(
      (encoder): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): ConvBlock(
          (conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=same)
            (1): LeakyReLU(negative_slope=0.1, inplace=True)
          )
        )
      )
    )
    (enc_2): Encoder(
      (encoder): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): ConvBlock(
     

In [6]:
inputs, labels = next(iter(current_model_dataloader))

In [12]:
inputs = inputs.to(device)
labels = labels.to(device)

optimizer.zero_grad()

outputs = model(inputs.float()) #weird datatype mismatching... for some reason it's seeing HRRR data as double
loss = loss_function(outputs,labels)
loss.backward()
optimizer.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([768, 2, 180, 180])) that is different to the input size (torch.Size([768, 1, 180, 180])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [8]:
outputs.size()

torch.Size([768, 2, 180, 180])